## Cas d’étude 1 : déterminants de l’état de pauvreté

## Importation des libraries

In [1]:
import numpy as np
import pandas as pd

## 1-Importation du dataset

In [2]:
dataset = pd.read_csv('impactpauv.csv', encoding ='ISO-8859-1')
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

In [3]:
print(X)

[[1.00000000e+00 5.00000000e+00 2.00000000e+00 ... 1.36500000e+03
  1.00000000e+00 0.00000000e+00]
 [1.00000000e+00 6.00000000e+00 2.00000000e+00 ... 1.16480005e+03
  2.00000000e+00 0.00000000e+00]
 [1.00000000e+00 4.00000000e+00 2.00000000e+00 ... 1.36500000e+03
  3.00000000e+00 0.00000000e+00]
 ...
 [2.00000000e+00 9.00000000e+00 2.00000000e+00 ... 1.58469995e+03
  4.87800000e+03 0.00000000e+00]
 [2.00000000e+00 1.30000000e+01 2.00000000e+00 ... 1.15959998e+03
  4.87900000e+03 0.00000000e+00]
 [2.00000000e+00 5.00000000e+00 2.00000000e+00 ... 1.58469995e+03
  4.88000000e+03 0.00000000e+00]]


In [6]:
print(y)
dataset.shape

[11684.40039062  8310.         20935.75       ...  3856.66674805
   828.53845215  6542.20019531]


(4880, 24)

## 2-Encodage des variables catégorielles

### Extraction des variables à utiliser

In [ ]:
 df =dataset[[ 'milieu', 'taille','sexecm', 'agecm', 'occup_cm', 'emcm', 'typlog', 'stocp', 'pauvre']]
 X2 = df.iloc[:, :-1].values
 y2 = df.iloc[:, -1].values

In [ ]:
print(X2)

[[ 1  5  1 ...  1  1  3]
 [ 1  6  1 ...  1  1  3]
 [ 1  4  1 ...  1  1  3]
 ...
 [ 2  9  1 ...  1  6  1]
 [ 2 13  1 ...  1  6  1]
 [ 2  5  1 ...  1  6  7]]


In [ ]:
print(y2)

[2 2 2 ... 2 2 2]


### Encodage de la variable indépendante

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0])], remainder='passthrough')
X2 = np.array(ct.fit_transform(X2))

In [ ]:
print(X2)

[[ 1.  0.  5. ...  1.  1.  3.]
 [ 1.  0.  6. ...  1.  1.  3.]
 [ 1.  0.  4. ...  1.  1.  3.]
 ...
 [ 0.  1.  9. ...  1.  6.  1.]
 [ 0.  1. 13. ...  1.  6.  1.]
 [ 0.  1.  5. ...  1.  6.  7.]]


### Encodage de la variable dépendante

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y2 = le.fit_transform(y2)

In [ ]:
print(y2)

[1 1 1 ... 1 1 1]


## 3-Répartition de la base de données en sous base de données (Entrainement/test)

In [ ]:
from sklearn.model_selection import train_test_split
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size = 0.2, random_state = 0)
print(X2_train)

[[1. 0. 8. ... 1. 3. 3.]
 [1. 0. 5. ... 1. 0. 3.]
 [1. 0. 3. ... 1. 2. 1.]
 ...
 [1. 0. 4. ... 1. 3. 1.]
 [1. 0. 4. ... 1. 3. 1.]
 [1. 0. 4. ... 1. 2. 1.]]


## 4-Modele de regression logistique

### Mise à echelle


In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X2_train = sc.fit_transform(X2_train)
X2_test = sc.transform(X2_test)
print(X2_train)

[[ 0.87687917 -0.87687917  0.72869297 ...  0.44693864 -0.65699767
   0.39598906]
 [ 0.87687917 -0.87687917 -0.32820512 ...  0.44693864 -2.17770057
   0.39598906]
 [ 0.87687917 -0.87687917 -1.03280385 ...  0.44693864 -1.16389863
  -0.54954079]
 ...
 [ 0.87687917 -0.87687917 -0.68050448 ...  0.44693864 -0.65699767
  -0.54954079]
 [ 0.87687917 -0.87687917 -0.68050448 ...  0.44693864 -0.65699767
  -0.54954079]
 [ 0.87687917 -0.87687917 -0.68050448 ...  0.44693864 -1.16389863
  -0.54954079]]


## Entraînement du modèle de la regression logistique

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X2_train, y2_train)
#classifier.get_params()
coef = classifier.fit(X2_train, y2_train).coef_
intecept = classifier.fit(X2_train, y2_train).intercept_
print(coef, intecept)

[[ 0.19898914 -0.19898914  0.04933146 -0.00441109  0.01053882 -0.03404245
  -0.0712993  -0.13105694 -0.04399926]] [1.83103714]


## 5- Formation de la matrice de confusion et calculer l’accuracy et l’AUC du modèle pour les 2 seuils 0.5 (par defaut) et 0.8

In [ ]:
y2_pred = classifier.predict_proba(X2_test)
#print(y2_pred)
y2_pred_df = pd.DataFrame(y2_pred[: ,1])
#print(y2_pred_df)
y2_test_pred = y2_pred_df.applymap(lambda x: 1 if x>=0.3 else 0)
#print(np.concatenate((y2_test_pred, y2_test.reshape(len(y2_test),1)),1))

from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score
cm = confusion_matrix(y2_test, y2_test_pred)
print(cm)
print(accuracy_score(y2_test, y2_test_pred))
print(roc_auc_score(y2_test, y2_test_pred))

[[0.24848258 0.75151742]
 [0.214051   0.785949  ]
 [0.09352233 0.90647767]
 ...
 [0.07138877 0.92861123]
 [0.2277794  0.7722206 ]
 [0.2224453  0.7775547 ]]
[[  0 132]
 [  0 844]]
0.8647540983606558
0.5
